In [1]:
import webotsgym as wg

from webotsgym.config import WebotConfig, SimSpeedMode
from webotsgym.environment import webotsgym, WebotsGrid
from webotsgym.evaluate import Evaluate, EvaluateMats, EvaluatePJ0
from webotsgym.action import DiscreteAction, ContinuousAction
from webotsgym.observation import Observation

import numpy as np

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
def exponential_decay(x, N0=1, lambda_=5):
    return N0*np.exp(-lambda_*x)

def exponential_penalty(x, step_penalty=-1, lambda_=5):
    return step_penalty * (1 - exponential_decay(x, lambda_=lambda_))

class MyEval(Evaluate):
    def __init__(self, env, config: WebotConfig = WebotConfig()):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        if self.env.get_target_distance(normalized=False) < 0.05:
            reward = 10000
        else:
            distance_normalized = self.env.get_target_distance(normalized=True)
            step_base_penalty = -1
            reward = exponential_penalty(x=distance_normalized, step_penalty=step_base_penalty, lambda_=3)
            if self.env.gps_visited_count > 3:
                reward -= 0.2 * (self.env.gps_visited_count - 2)**2
            if self.env.state._action_denied is True:
                reward -= 500
        return reward
        
        
    def check_done(self):
        if self.env.time_steps == 300:
            return True
        if self.env.total_reward < -10000:
            return True
        if self.env.get_target_distance(normalized=False) < 0.05:
            return True
        return False


config = WebotConfig()
config.sim_mode = SimSpeedMode.FAST
config.reset_env_after = 20000
config.num_obstacles = 12
config.world_size = 8
env = WebotsGrid(train=True, 
                 config=config,
                 evaluate_class = MyEval)

/home/jonas/anaconda3/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Accepting on Port:  10201
sending: start env 2


In [3]:
time_steps = 0
model_name = "PPO_webots_v1_jonas_no_while"

model = PPO1("MlpPolicy", env, verbose=1, tensorboard_log="./PPO_webots_v1_jonas_no_while_tensorboard/")
model.learn(total_timesteps=50000)
model.save("models/keep/{}".format(model_name))

#while time_steps < 500000:
#    model = PPO1("MlpPolicy", env, verbose=1)
#    model.load("models/keep/{}".format(model_name))
#    model.learn(total_timesteps=10000)
#    model.save("models/keep/{}".format(model_name))
#    time_steps += 10000





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

********** Iteration 0 ************
sending: start env 2
sending: start env 2
sending: start env 2

Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00141 |      -0.01386 |      3.71e+06 |       0.00066 |       1.38555
     -0.01244 |      -0.01383 |      3.71e+06 |       0.00362 |       1.38252
     -0.01612 |      -0.01377 |      3.71e+06 |       0.00862 |       1.37747
     -0.01950 |      -0.01375 |      3.71e+06 |       0.01110 |       1.37506
Evaluating losses...
     -0.02253 |      -0.01376 |      3.71e+06 |       0.01054 |       1.37565
----------------------------------
| EpLenMean       | 40           |
| EpRewMean       | 9.64e+03     |
| EpThisIter      | 2            |
| EpisodesSoFar 

********** Iteration 4 ************
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00053 |      -0.01375 |      4.30e+06 |      5.00e-05 |       1.37537
     -0.00475 |      -0.01373 |      4.30e+06 |       0.00058 |       1.37316
     -0.01742 |      -0.01370 |      4.30e+06 |       0.00274 |       1.36991
     -0.01956 |      -0.01364 |      4.30e+06 |       0.00815 |       1.36398
Evaluating losses...
     -0.01858 |      -0.01361 |      4.30e+06 |       0.01120 |       1.36122
----------------------------------
| EpLenMean       | 112          |
| EpRewMean       | 6.45e+03     |
| EpThisIter      | 4            |
| EpisodesSoFar   | 10           |
| TimeElapsed     | 164          |
| TimestepsSoFar  | 1280         |
| ev_tdlam_before | 0.000102     |
| loss_ent        | 1.3612224    |
| loss_kl         | 0.011197823  |
| loss_pol_entpen | -0.0136

********** Iteration 12 ************
sending: start env 2
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00098 |      -0.01363 |      6.95e+04 |       0.00014 |       1.36273
     -0.00702 |      -0.01363 |      6.94e+04 |       0.00145 |       1.36326
     -0.01741 |      -0.01362 |      6.92e+04 |       0.00498 |       1.36198
     -0.02615 |      -0.01358 |      6.89e+04 |       0.01042 |       1.35760
Evaluating losses...
     -0.02820 |      -0.01354 |      6.87e+04 |       0.01406 |       1.35437
----------------------------------
| EpLenMean       | 144          |
| EpRewMean       | 3.82e+03     |
| EpThisIter      | 1            |
| EpisodesSoFar   | 23           |
| TimeElapsed     | 402          |
| TimestepsSoFar  | 3328         |
| ev_tdlam_before | 0.0142       |
| loss_ent        | 1.3543742    |
| loss_kl         | 0.014061857  |
| loss_pol_entpen | -0.013543742 |
| loss_pol_surr   | -0.028204221 |
| loss_vf_loss    |

********** Iteration 20 ************
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00243 |      -0.01347 |      1.93e+07 |       0.00029 |       1.34677
     -0.01473 |      -0.01346 |      1.92e+07 |       0.00394 |       1.34608
     -0.02335 |      -0.01340 |      1.92e+07 |       0.01275 |       1.34011
     -0.02885 |      -0.01337 |      1.92e+07 |       0.01553 |       1.33727
Evaluating losses...
     -0.03424 |      -0.01337 |      1.92e+07 |       0.01392 |       1.33725
----------------------------------
| EpLenMean       | 108          |
| EpRewMean       | 5.42e+03     |
| EpThisIter      | 9            |
| EpisodesSoFar   | 49           |
| TimeElapsed     | 731          |
| TimestepsSoFar  | 5376         |
| ev_tdlam_before | 0.0001

sending: start env 2
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00035 |      -0.01283 |      1.31e+07 |      3.75e-05 |       1.28336
     -0.00743 |      -0.01279 |      1.31e+07 |       0.00064 |       1.27923
     -0.01846 |      -0.01272 |      1.31e+07 |       0.00429 |       1.27163
     -0.02327 |      -0.01266 |      1.31e+07 |       0.00870 |       1.26636
Evaluating losses...
     -0.02605 |      -0.01266 |      1.31e+07 |       0.00960 |       1.26640
----------------------------------
| EpLenMean       | 83.2         |
| EpRewMean       | 6.79e+03     |
| EpThisIter      | 4            |
| EpisodesSoFar   | 86           |
| TimeElapsed     | 1.12e+03     |
| TimestepsSoFar  | 7168         |
| ev_tdlam_before | -5.56e-05    |
| loss_ent        | 1.2663956    |
| loss_kl         | 0.009598469  |
| loss_pol_entpen | -0.012663955 |
| loss_pol_surr   | -0.026052859 |
| loss_vf_loss    | 13112116.0   |
---------------------

...

sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
    -9.61e-05 |      -0.00945 |      2.35e+07 |      1.71e-05 |       0.94510
     -0.00091 |      -0.00946 |      2.35e+07 |       0.00011 |       0.94626
     -0.00179 |      -0.00947 |      2.35e+07 |       0.00016 |       0.94661
     -0.00306 |      -0.00946 |      2.35e+07 |       0.00012 |       0.94604
Evaluating losses...
     -0.00382 |      -0.00945 |      2.35e+07 |      9.80e-05 |       0.94528
----------------------------------
| EpLenMean       | 17.9         |
| EpRewMean       | 8.59e+03     |
| EpThisIter      | 15           |
| EpisodesSoFar   | 1712         |
| TimeElapsed     | 2.3e+04      |
| TimestepsSoFar  | 43776        |
| ev_tdlam_before | 0.0221       |
| loss_ent        | 0.94527864   |
| loss_kl     

sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00040 |      -0.00802 |      4.09e+07 |      3.64e-06 |       0.80183
     -0.00174 |      -0.00801 |      4.09e+07 |      5.44e-05 |       0.80076
     -0.00237 |      -0.00800 |      4.09e+07 |       0.00012 |       0.80005
     -0.00270 |      -0.00800 |      4.09e+07 |       0.00013 |       0.80021
Evaluating losses...
     -0.00340 |      -0.00801 |      4.09e+07 |       0.00011 |       0.80082
-----------------------------------
| EpLenMean       | 15.7          |
| EpRewMean       | 9.19e+03      |
| EpThisIter      | 21            |
| EpisodesSoFar   | 1805          |
| TimeElapsed     | 2.37e+04      |
| TimestepsSoFar  | 45312         |
| ev_tdlam_bef

********** Iteration 182 ************
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00017 |      -0.01149 |      5.42e+06 |      1.74e-06 |       1.14899
    -3.55e-05 |      -0.01149 |      5.42e+06 |      1.04e-05 |       1.14879
     -0.00057 |      -0.01149 |      5.42e+06 |      2.04e-05 |       1.14872
     -0.00071 |      -0.01149 |      5.42e+06 |      3.04e-05 |       1.14871
Evaluating losses...
     -0.00084 |      -0.01149 |      5.42e+06 |      3.11e-05 |       1.14882
------------------------------------
| EpLenMean       | 15.1           |
| EpRewMean       | 9.33e+03       |
| EpThisIter      | 4              |
| EpisodesSoFar   | 1896           |
| TimeElapsed     | 2.44e+04       |
| TimestepsSoFar  | 46848          |
| ev_tdlam_before | 0.00244        |
| loss_ent        | 1.1488211      |
| loss_kl         | 3

     -0.00089 |      -0.01044 |      1.30e+07 |      2.49e-05 |       1.04401
Evaluating losses...
     -0.00115 |      -0.01044 |      1.30e+07 |      1.71e-05 |       1.04380
-----------------------------------
| EpLenMean       | 21.1          |
| EpRewMean       | 8.74e+03      |
| EpThisIter      | 7             |
| EpisodesSoFar   | 1964          |
| TimeElapsed     | 2.5e+04       |
| TimestepsSoFar  | 48384         |
| ev_tdlam_before | 0.0177        |
| loss_ent        | 1.0438018     |
| loss_kl         | 1.7075607e-05 |
| loss_pol_entpen | -0.010438018  |
| loss_pol_surr   | -0.0011461545 |
| loss_vf_loss    | 13033821.0    |
-----------------------------------
********** Iteration 189 ************
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending:

********** Iteration 195 ************
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
sending: start env 2
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     4.64e-07 |      -0.01115 |      4.61e+06 |     -2.28e-09 |       1.11545
    -1.47e-05 |      -0.01115 |      4.61e+06 |     -1.06e-09 |       1.11545
    -1.99e-05 |      -0.01115 |      4.61e+06 |      8.98e-10 |       1.11545
    -2.33e-05 |      -0.01115 |      4.61e+06 |      8.07e-09 |       1.11545
Evaluating losses...
    -2.70e-05 |      -0.01115 |      4.61e+06 |      8.49e-09 |       1.11545
------------------------------------
| EpLenMean       | 21.1           |
| EpRewMean       | 8.7e+03        |
| EpThisIter      | 7              |
| EpisodesSoFar   | 2051           |
| TimeElapsed     | 2.57e+04       |
| TimestepsSoFar  | 50176          |
| ev_tdlam_before | 0.021          |
| loss_ent      

In [4]:
int(config.fast_simulation)

AttributeError: 'WebotConfig' object has no attribute 'fast_simulation'